In [6]:
# ================================
# Google Drive & Dataset unzip
# ================================
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!unzip -q /content/drive/MyDrive/Final_Dataset.zip -d /content/

In [8]:
# ================================
# YOLOv8 (Classification) Setup
# ================================

!pip install ultralytics


In [9]:
import os, shutil, glob, time
import torch
from ultralytics import YOLO

In [11]:
# ==========================================================
# ================ CONFIGURATION PARAMETERS ================
# ==========================================================
DATA_DIR = '/content/Final_Dataset'  # expects subfolders: train/, validation/ (or val/), test/ (optional)
RUNS_PROJECT = 'runs/classify'
RUN_NAME = 'grocery_cls_v1'
MODEL_SAVE_PATH = '/content/drive/MyDrive/grocery_yolov8cls.pt'  # final copy to Drive

NUM_EPOCHS = 25        # keep same as your ResNet script
BATCH_SIZE = 32
IMG_SIZE = 224         # common for cls; 224–320 works well
DEVICE = '0' if torch.cuda.is_available() else 'cpu'  # GPU if available

In [12]:
# ==========================================================
# ============== DATASET FOLDER NORMALISATION =============
# ==========================================================
train_dir = os.path.join(DATA_DIR, 'train')
val_dir_src = os.path.join(DATA_DIR, 'validation')
val_dir = os.path.join(DATA_DIR, 'val')

if os.path.isdir(val_dir_src) and not os.path.exists(val_dir):
    os.symlink(val_dir_src, val_dir)

# Verify class folders exist under train
assert os.path.isdir(train_dir), "train/ folder not found inside Final_Dataset"
assert os.path.isdir(val_dir) or os.path.isdir(val_dir_src), "val/ or validation/ folder not found"

# (Optional) Show detected classes from train subfolders
classes = sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])
print(f"Detected classes ({len(classes)}): {classes}")

Detected classes (6): ['bottle', 'bread', 'can', 'detergent', 'pulses', 'vegetable']


In [13]:
# ==========================================================
# ================ LOAD PRETRAINED YOLOv8-CLS ==============
# ==========================================================
model = YOLO('yolov8n-cls.pt')

In [14]:
# ==========================================================
# =================== TRAIN YOLOv8-CLS =====================
# ==========================================================
# This replaces the manual training loop. Ultralytics handles:
# - augmentations
# - class weighting (implicitly via sampling & loss)
# - metrics (top1/top5)
# - best/last checkpoints
print("\n[YOLOv8-CLS] Starting training...")
start_time = time.time()

results = model.train(
    data=DATA_DIR,          # Root folder that contains train/ and val/
    epochs=NUM_EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=DEVICE,
    project=RUNS_PROJECT,
    name=RUN_NAME,
    pretrained=True,        # use ImageNet pretraining
    verbose=True,
)

elapsed = time.time() - start_time
print(f"\n--- Training Complete in {elapsed//60:.0f}m {elapsed%60:.0f}s ---")

# Locate best weights (Ultralytics always saves to runs/classify/<name>/weights/best.pt)
save_dir = results.save_dir if hasattr(results, 'save_dir') else os.path.join(RUNS_PROJECT, RUN_NAME)
best_weights = os.path.join(str(save_dir), 'weights', 'best.pt')
print(f"Best model: {best_weights}")

# Copy best weights to Drive with your preferred filename
shutil.copy(best_weights, MODEL_SAVE_PATH)
print(f"Copied best weights to: {MODEL_SAVE_PATH}")


[YOLOv8-CLS] Starting training...
Ultralytics 8.3.226 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Final_Dataset, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=grocery_cls_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=1

In [15]:
# ==========================================================
# ===================== EVALUATION =========================
# ==========================================================
# We validate on "test/" if present; otherwise Ultralytics defaults to val split.
print("\n[YOLOv8-CLS] Evaluating on test/ (if available)...")
split_to_use = 'test' if os.path.isdir(os.path.join(DATA_DIR, 'test')) else 'val'
metrics = model.val(
    data=DATA_DIR,
    split=split_to_use,
    imgsz=IMG_SIZE,
    device=DEVICE
)
# metrics dict includes top1/top5 accuracy
try:
    print(f"Top-1 Acc: {metrics.top1:.4f} | Top-5 Acc: {metrics.top5:.4f}")
except Exception:
    print("Validation complete. (Metrics object printed below)")
    print(metrics)


[YOLOv8-CLS] Evaluating on test/ (if available)...
Ultralytics 8.3.226 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,442,566 parameters, 0 gradients, 3.3 GFLOPs
train: /content/Final_Dataset/train... found 2925 images in 6 classes ✅ 
val: /content/Final_Dataset/val... found 624 images in 6 classes ✅ 
test: /content/Final_Dataset/test... found 634 images in 6 classes ✅ 
test: Fast image access ✅ (ping: 0.0±0.0 ms, read: 473.8±296.2 MB/s, size: 19.6 KB)
test: Scanning /content/Final_Dataset/test... 634 images, 0 corrupt: 100% ━━━━━━━━━━━━ 634/634 6.6Kit/s 0.1s
test: New cache created: /content/Final_Dataset/test.cache
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 40/40 17.4it/s 2.3s
                   all      0.997          1
Speed: 0.2ms preprocess, 1.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/runs/classify/val
Top-1 Acc: 0.9968 | Top-5 Acc: 1.0000
